# W7L1 IP Logical Constraints - In Class Fixed Cost Planes

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Model Definition](#3)<br>
4. [Model Solution](#4)<br>
5. [Redo Model with new constraints](#5)<br>

## 1. Problem Statement<a id = 1></a>

Here we will solve the problem where we need to determine how many of each plane to produce. Each plane has a setup cost and our goal is to maximize our project. 

##### [Back to Top](#Top)

## 2. Data<a id = 2></a>

The data are the sheet Planes tab in w08-c02-planes.xlsx.

In [1]:
import pandas as pd
import pyomo.environ as pe
#import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_excel('w08-c02-planes.xlsx', sheet_name = 'Planes')
raw_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Part One
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Rocket,Meteor,Streak,Comet,Jet,Biplane,Available,NaN,NaN
2,Steel,1,4,0,4,2,1,800,NaN,NaN
3,Copper,4,5,3,0,1,0,1160,NaN,NaN
4,Plastic,0,3,8,0,1,0,1780,NaN,NaN
5,Rubber,2,0,1,2,1,5,1050,NaN,NaN
6,Glass,2,4,2,2,2,4,1360,NaN,NaN
7,Paint,1,4,1,4,3,4,1240,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Profit,30,45,24,26,24,30,NaN,NaN,NaN


First we will capture the coefficients which will the the profit per each plane produced, the setup fixed cost, and the units of each mataerial needed for each plane type.

In [4]:
DV_indexes = ['Rocket', 'Meteor', 'Streak', 'Comet', 'Jet', 'Biplane']
coef = pd.DataFrame(raw_data.iloc[[9, 10, 2, 3, 4, 5, 6, 7], range(1, 7)])
coef.index = ['profit', 'Fixed Setup', 'Steel', 'Copper', 'Plastic', 'Rubber', 'Glass', 'Paint']
coef.columns = DV_indexes
coef

,Rocket,Meteor,Streak,Comet,Jet,Biplane
profit,30,45,24,26,24,30
Fixed Setup,35,20,60,70,75,30
Steel,1,4,0,4,2,1
Copper,4,5,3,0,1,0
Plastic,0,3,8,0,1,0
Rubber,2,0,1,2,1,5
Glass,2,4,2,2,2,4
Paint,1,4,1,4,3,4


The next tables will give us our max values for our constraints - we have units capacity for each raw material.

In [5]:
hours = pd.DataFrame(raw_data.iloc[2:8, 7])
hours.columns = ['units']
hours.index = coef.index[2:]
hours

,units
Steel,800
Copper,1160
Plastic,1780
Rubber,1050
Glass,1360
Paint,1240


##### [Back to Top](#Top)

## 3. Model Definition<a id = 3></a>

Now let's solve the model.

In [28]:
model = pe.ConcreteModel()

### Define Decision Variables

__NOTE:__ for this model you have 2 sets of changing cells that you want the solver to determine - the `x`plane quantities and the `yU` values which are binary utilization variables we will used to create the fixed setup cost linking constraints. Let's make the `x` values `NonNegativeIntegers` since we can't make a fraction of a plane.

In [29]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeIntegers)
model.y = pe.Var(DV_indexes, domain = pe.Binary)
model.x.pprint()
model.y.pprint()

x : Size=6, Index=x_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    Biplane :     0 :  None :  None : False :  True : NonNegativeIntegers
      Comet :     0 :  None :  None : False :  True : NonNegativeIntegers
        Jet :     0 :  None :  None : False :  True : NonNegativeIntegers
     Meteor :     0 :  None :  None : False :  True : NonNegativeIntegers
     Rocket :     0 :  None :  None : False :  True : NonNegativeIntegers
     Streak :     0 :  None :  None : False :  True : NonNegativeIntegers
y : Size=6, Index=y_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    Biplane :     0 :  None :     1 : False :  True : Binary
      Comet :     0 :  None :     1 : False :  True : Binary
        Jet :     0 :  None :     1 : False :  True : Binary
     Meteor :     0 :  None :     1 : False :  True : Binary
     Rocket :     0 :  None :     1 : False :  True : Binary
     Streak :     0 :  None :     1 : False :  True : Binary


### Define objective function

Note that this will be the total profit which is calculated from the fixed price and variable setup costs. Make sure you see how these are calculated in the Excel sheet before you try to implement here.

In [30]:
#obj funct max profit * x - setup cost * y
model.obj = pe.Objective(expr = sum(coef.loc['profit', index]*model.x[index] 
                                    for index in DV_indexes) - 
                         sum(coef.loc['Fixed Setup', index]*model.y[index] for index in DV_indexes),
                         sense = pe.maximize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 30*x[Rocket] + 45*x[Meteor] + 24*x[Streak] + 26*x[Comet] + 24*x[Jet] + 30*x[Biplane] - (35*y[Rocket] + 20*y[Meteor] + 60*y[Streak] + 70*y[Comet] + 75*y[Jet] + 30*y[Biplane])


### Define Constraints

In [31]:
#Units Capacity Constraints
model.Steel = pe.Constraint(expr = sum(coef.loc['Steel', index]*model.x[index]
                                       for index in DV_indexes) <= hours.loc['Steel', 'units'])
model.Copper = pe.Constraint(expr = sum(coef.loc['Copper', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Copper', 'units'])
model.Plastic = pe.Constraint(expr = sum(coef.loc['Plastic', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Plastic', 'units'])
model.Rubber = pe.Constraint(expr = sum(coef.loc['Rubber', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Rubber', 'units'])
model.Glass = pe.Constraint(expr = sum(coef.loc['Glass', index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Glass', 'units'])
model.Paint = pe.Constraint(expr = sum(coef.loc['Paint',index]*model.x[index] 
                                       for index in DV_indexes) <= hours.loc['Paint', 'units'])

#Linking Constraints with really large arbitrary max demand of 2000
model.LinkRocket = pe.Constraint(expr = model.x['Rocket'] <= 2000*model.y['Rocket'])
model.LinkMeteor = pe.Constraint(expr = model.x['Meteor'] <= 2000*model.y['Meteor'])
model.LinkStreak = pe.Constraint(expr = model.x['Streak'] <= 2000*model.y['Streak'])
model.LinkComet = pe.Constraint(expr = model.x['Comet'] <= 2000*model.y['Comet'])
model.LinkJet = pe.Constraint(expr = model.x['Jet'] <= 2000*model.y['Jet'])
model.LinkBiplane = pe.Constraint(expr = model.x['Biplane'] <= 2000*model.y['Biplane'])

for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

Steel : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                         : Upper : Active
    None :  -Inf : x[Rocket] + 4*x[Meteor] + 4*x[Comet] + 2*x[Jet] + x[Biplane] : 800.0 :   True
Steel None
Copper : Size=1, Index=None, Active=True
    Key  : Lower : Body                                             : Upper  : Active
    None :  -Inf : 4*x[Rocket] + 5*x[Meteor] + 3*x[Streak] + x[Jet] : 1160.0 :   True
Copper None
Plastic : Size=1, Index=None, Active=True
    Key  : Lower : Body                               : Upper  : Active
    None :  -Inf : 3*x[Meteor] + 8*x[Streak] + x[Jet] : 1780.0 :   True
Plastic None
Rubber : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                         : Upper  : Active
    None :  -Inf : 2*x[Rocket] + x[Streak] + 2*x[Comet] + x[Jet] + 5*x[Biplane] : 1050.0 :   True
Rubber None
Glass : Size=1, Index=None, Active=True
    Key  : Lower : Body                   

##### [Back to Top](#Top)

## 4. Model Solution<a id = 4></a>

In [32]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

ok optimal


### Optimal Objective Value

In [33]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $14764.00


### Optimal Decision Variables

In [34]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

,Rocket,Meteor,Streak,Comet,Jet,Biplane
x,96.0,0.0,195.0,0.0,191.0,94.0
y,1.0,0.0,1.0,0.0,1.0,1.0


##### [Back to Top](#Top)

## 5. Now solve the model with the Logical and Disjunctive Constraints<a id=5></a>

Now we want to add the new constraints:
* Suppose that we must pick the Meteor or Comet in our production.
* Suppose also that Rocket must be > 100 or Biplane must be > 100
    

In [35]:
#NEW BINARY Rocket > 100 Indicator
model.yRocketConst = pe.Var(domain = pe.Binary)
#NEW BINARY Biplane > 100 Indicator
model.yBiplaneConst = pe.Var(domain = pe.Binary)

#New Logical constraint - pick the Meteor or Comet
model.MetORCom = pe.Constraint(expr = model.y['Meteor']+model.y['Comet'] >= 1)

#New Disjunctive constraint - Rocket must be > 100 or Biplane must be > 100
#Rocket−𝑀*yRocket≤100 and Rocket+𝑀(1−𝑦Rocket)≥100
max_dem = 2000
model.RockConst1 = pe.Constraint(expr = model.x['Rocket']-max_dem*model.yRocketConst <= 100)
model.RockConst2 = pe.Constraint(expr = model.x['Rocket']+max_dem*(1-model.yRocketConst) >=100)
#Biplane−𝑀*yBiplane≤100 and Biplane+𝑀(1−𝑦Biplane)≥100
model.BipConst1 = pe.Constraint(expr = model.x['Biplane']-max_dem*model.yBiplaneConst <= 100)
model.BipConst2 = pe.Constraint(expr = model.x['Biplane']+max_dem*(1-model.yBiplaneConst) >=100)
                                 
#Binary var YRocket Const + Binary var YBiplane Const >= 1
model.RockBipLink = pe.Constraint(expr = model.yRocketConst+model.yBiplaneConst >= 1)
model.pprint()

2 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Rocket', 'Meteor', 'Streak', 'Comet', 'Jet', 'Biplane'}
    y_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Rocket', 'Meteor', 'Streak', 'Comet', 'Jet', 'Biplane'}

4 Var Declarations
    x : Size=6, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
        Biplane :     0 :  94.0 :  None : False : False : NonNegativeIntegers
          Comet :     0 :   0.0 :  None : False : False : NonNegativeIntegers
            Jet :     0 : 191.0 :  None : False : False : NonNegativeIntegers
         Meteor :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Rocket :     0 :  96.0 :  None : False : False : NonNegativeIntegers
         Streak :     0 : 195.0 :  None : False : False : NonNegativeIntegers
    y 

In [36]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

ok optimal


### New Optimal Objective Value

In [37]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $14740.00


### New Optimal Decision Variables

In [38]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

,Rocket,Meteor,Streak,Comet,Jet,Biplane,NaN
x,116.0,0.0,215.0,131.0,51.0,58.0,NaN
y,1.0,0.0,1.0,1.0,1.0,1.0,NaN
yRocketConst,NaN,NaN,NaN,NaN,NaN,NaN,1.0
yBiplaneConst,NaN,NaN,NaN,NaN,NaN,NaN,0.0


##### [Back to Top](#Top)

In [50]:
for con in model.component_objects(pe.Constraint):
    print(con.lower, con.slack(), con.upper)

None 0.0 800.0
None 0.0 1160.0
None 9.0 1780.0
None 0.0 1050.0
None 102.0 1360.0
None 0.0 1240.0
None 1884.0 0.0
None 0.0 0.0
None 1785.0 0.0
None 1869.0 0.0
None 1949.0 0.0
None 1942.0 0.0
1.0 0.0 None
None 1984.0 100.0
100.0 16.0 None
None 42.0 100.0
100.0 1958.0 None
1.0 0.0 None
